# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
import requests
import time
from scipy.stats import linregress
import random

# Import the API key
from api_keys import weather_api_key

# citipy to determine cities based latitude and longitude
from citipy import citipy


## Generate Cities List

In [2]:
# define list that will hold the city names
cities = []

# generate random cities list by looping through random coordinates
for x in range(30):
    
    # generate random latitude and longitude coordinates
    r_lat = random.randint(-90,90)
    r_lng = random.randint(-180,180)
    
    # generate cities and append list based on randomised lat,lng.
    city = citipy.nearest_city(r_lat,r_lng)
    cities.append(city.city_name)

# delete duplicates with set function
cities = list(set(cities))
len(cities)

28

In [3]:
type(cities)

list

In [4]:
cities[:10]

['saint-francois',
 'punta arenas',
 'chapais',
 'cabedelo',
 'geraldton',
 'ushuaia',
 'kavaratti',
 'tuktoyaktuk',
 'belushya guba',
 'airai']

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [6]:
# Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"

# Build query URL
query_url = url + "appid=" + weather_api_key + "&units=" + units + "&q="

# Set up lists to hold reponse info
city = []
lat = []
lng = []
max_temp = []
humidity = []
cloudiness = []
wind_speed = [] 
country = []
date = []

# Set start values
records = 0
sets = 1

# Print statements at beginning 
print("Beginning Data Retrieval")    
print("-----------------------------")

# Loop through the list of cities and perform a request for data for each
for y in cities:
    # request data for each city
    response = requests.get(query_url + y).json()
    
    # Records go up to 50
    if records < 50:
        records += 1
        
    # Sets increase after record 50
    else:
        sets += 1
        records = 0
        
    # Print statement has to go here or record numbers skip when city not found. Not sure why
    print(f"Processing Record {records} of Set {sets} | {y}")

    try:
        city.append(response["name"])
        lat.append(response["coord"]["lat"])
        lng.append(response["coord"]["lon"])
        max_temp.append(response["main"]["temp_max"])
        humidity.append(response["main"]["humidity"])
        cloudiness.append(response["clouds"]["all"])
        wind_speed.append(response["wind"]["speed"])
        country.append(response["sys"]["country"])
        date.append(response["dt"])
                          
    except KeyError:
        print("City not found. Skipping...")

# Print statements at end
print("-----------------------------")
print("Data Retrieval Complete")      
print("-----------------------------")

Beginning Data Retrieval
-----------------------------
Processing Record 1 of Set 1 | saint-francois
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | chapais
Processing Record 4 of Set 1 | cabedelo
Processing Record 5 of Set 1 | geraldton
Processing Record 6 of Set 1 | ushuaia
Processing Record 7 of Set 1 | kavaratti
Processing Record 8 of Set 1 | tuktoyaktuk
Processing Record 9 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 10 of Set 1 | airai
Processing Record 11 of Set 1 | kodiak
Processing Record 12 of Set 1 | mataura
Processing Record 13 of Set 1 | vaini
Processing Record 14 of Set 1 | yellowknife
Processing Record 15 of Set 1 | stepankovice
Processing Record 16 of Set 1 | kapaa
Processing Record 17 of Set 1 | muros
Processing Record 18 of Set 1 | bang lamung
Processing Record 19 of Set 1 | hofn
Processing Record 20 of Set 1 | bluff
Processing Record 21 of Set 1 | lolua
City not found. Skipping...
Processing Record 22 of Set 1 | ri

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression